In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D,Dense,Dropout,Input,
from tensorflow.keras.layers import Dense,Dropout,MaxPooling2D,Conv2D,GlobalAveragePooling2D,RandomFlip,RandomRotation,RandomBrightness,RandomContrast,RandomZoom,Input,Rescaling
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau,TensorBoard
from PIL import Image
import os

In [2]:
classnames=[]
os.listdir()

['project_now.ipynb', 'train', 'validation']

In [ ]:
img_skip=0
for folder_name in classnames:
    folder_path=os.path.join("train",folder_name)
    print(folder_path)
    for f_name in os.listdir(folder_path):
        f_path =os.path.join(folder_path,f_name)
        try:
            img_bytes=tf.io.read_file(f_path)
            tf.io.decode_image(img_bytes)
           
        except Exception as e:
            print(f"Bad file found and deleted on {f_path}")
            img_skip +=1
            os.remove(f_path)

print(f"picture deleted due to corrupt {img_skip}")

NameError: name 'classnames' is not defined

In [ ]:
data_aug=Sequential([
     RandomFlip('horizontal'),
    RandomRotation(0.3),
    RandomZoom(height_factor=0.3),
    RandomBrightness(0.2),
    RandomContrast(0.2)
])

In [ ]:
auto_tune=tf.data.AUTOTUNE
training_ds =train_data.cache().prefetch(buffer_size=auto_tune)
validation_ds =val_data.cache().prefetch(buffer_size=auto_tune)

In [ ]:
train_data =tf.keras.utils.image_dataset_from_directory(
    directory,
    subset='training',
    seed=2345,
    validation_split=0.2,
    image_size=(220,220),
    batch_size=20
)
val_data =tf.keras.utils.image_dataset_from_directory(
    directory,
    subset='validation',
    seed=2345,
    validation_split=0.2,
    image_size=(220,220),
    batch_size=20
)

In [ ]:
trained_model =tf.keras.applications.MobileNetV2(
  input_shape=(220,220,3),
  include_top=False,
  weights='imagenet'
  )

In [ ]:
trained_model.trainable=False

In [ ]:
model=Sequential([
    Input(shape=(220,220,3)),
    data_aug,
    Rescaling(1./127.5,offset=1),
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(124,activation='relu'),
    Dropout(0.3),
    Dense(4,activation='softmax')
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=[EarlyStopping(monitor='val_accuracy',patience=3,mode='max'),
               ModelCheckpoint('tomato_pest_best_model.keras',save_best_only=True,monitor='val_loss',mode='min'),
               ReduceLROnPlateau( monitor='val_loss',factor=0.2,patience=5,min_lr=1e-7,mode='min'),
               TensorBoard(log_dir='logs', histogram_freq=1,write_graph=True,write_images=False,update_freq='epoch',profile_batch=2,embeddings_freq=0,embeddings_metadata=None)]
)